In [2]:
from __future__ import annotations
import json


def decrypt_json(json_path: str) -> dict:
    with open(json_path) as f:
        return json.load(f)


class Route:
    """Edges"""

    def __init__(self, origin: str, destination: str, travelTime: int):
        self.origin = Planet(origin)
        self.destination = Planet(destination)
        self.travel_time = travelTime

    def __repr__(self):
        return f"{self.origin}, {self.destination}, {self.travel_time}"





class Falcon:
    def __init__(self, falcon_json_path: str):
        self.data = decrypt_json(falcon_json_path)
        self.autonomy = self.data["autonomy"]
        self.routes = self._build_routes(self.data["routes"])

    def _build_routes(self, routes_json: dict[list]) -> dict:
        routes = {}
        for route in routes_json:
            route = Route(**route)
            if route.origin not in routes:
                routes[route.origin] = {}
            routes[route.origin][route.destination] = route.travel_time
        return routes


F = Falcon("falcon.json")
F.routes


NameError: name 'Planet' is not defined

In [ ]:
class Planet:
    def __init__(self, name: str):
        self.name = name

    def __hash__(self):
        return hash(str(self))

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

    def __eq__(self, other: Planet):
        if isinstance(other, Planet):
            return self.name == other.name
        if isinstance(other, str):
            return self.name == other


In [ ]:
class Empire:
    def __init__(self, empire_json_path: str):
        self.data = decrypt_json(empire_json_path)
        self.countdown = self.data["countdown"]
        self.bounty_hunters = self._build_hunters_positions(self.data["bounty_hunters"])

    def _build_hunters_positions(self, hunters_list: list[dict]):
        positions = {}
        for hunter in hunters_list:
            planet, day = hunter.values()
            if planet not in positions:
                positions[planet] = []
            positions[planet].append(day)
        return positions


E = Empire("empire.json")


In [ ]:
class Trip:
    def __init__(self, falcon: Falcon, empire: Empire, start: Planet, end: Planet):
        self.path = [start]

        self.routes = falcon.routes
        self.autonomy = falcon.autonomy
        self.tank = falcon.autonomy
        self.bounty_hunters = empire.bounty_hunters
        self.countdown = empire.countdown
        self.start = start
        self.end = end
        self.day = 0
        self.days_with_hunters = 0

    @property
    def neighbors(self):
        return (
            self.routes[self.current_planet]
            if self.current_planet in self.routes
            else []
        )

    @property
    def current_planet(self):
        return self.path[-1]

    def are_hunters_here(self) -> bool:
        if self.current_planet in self.bounty_hunters:
            return self.day in self.bounty_hunters[self.current_planet]
        return False

    def is_tank_empty(self) -> bool:
        return self.tank == 0

    def is_out_of_time(self, travel_time: int = 0) -> bool:
        return self.day + travel_time > self.countdown

    def is_arrived(self) -> bool:
        return self.current_planet == self.end

    def refuel(self) -> None:
        self.tank = self.autonomy
        self.day += 1
        self.path += [self.current_planet]
        self.days_with_hunters += self.are_hunters_here()

    def travel(self, destination: Planet) -> None:
        if destination not in self.neighbors:
            raise ValueError("This destination is not a neighbor planet.")
        if self.is_arrived():
            raise ValueError("Already arrived!")
        distance = self.neighbors[destination]
        if self.is_out_of_time(distance):
            raise ValueError("Out of time!")
        if distance > self.tank:
            raise ValueError(
                "The tank is not filled enough to travel to the destination."
            )
        self.path += [destination]
        self.tank -= distance
        self.day += distance
        self.days_with_hunters += self.are_hunters_here()


t = Trip(F, E, "Tatooine", "Endor")
t.neighbors


{Dagobah: 6, Hoth: 6}

In [ ]:
class C3PO:
    def __init__(self, falcon: Falcon, empire: Empire):
        self.falcon = falcon
        self.empire = empire

    def trip_explorer(self, start: Planet, end: Planet):
        trip = Trip(self.falcon, self.empire, start, end)

        def _explorer(trip: Trip):
            if trip.is_arrived():
                

            for neighbor in trip.neighbors:
                try:
                    neighbor_trip = trip
                    neighbor_trip.travel(neighbor)
                    self._explorer(neighbor_trip)
                except:
                    del trip

            try:
                trip.refuel()
                self._explorer(trip)
            except:
                del trip
            
c3po = C3PO(F, E)
c3po.trip_explorer("D", "E")


False
